### **Import Dependencies**

In [1]:
import cv2
import numpy as np

### **Define path and read the image**

In [32]:
# define the path where our images save 
path = r"C:\Users\M.Rodriguez\OneDrive\Desktop\image\1.png"

# read the image and convert it to cv2 type
image = cv2.imread(path)

# our image is grayscale but have 3 color channel, so we just need channel one  
image = image[:,:,0]


num_row = image.shape[0]
num_col = image.shape[1]

### **Crop the image**

In [33]:
# save the number of rows and number of columns of a image
num_row = image.shape[0]
num_col = image.shape[1]

# crop the image (to better detect the line)
image = image[num_row - 50:num_row, 0: num_col]
image = np.ascontiguousarray(image, dtype=np.uint8)

### **Preprocessing the image**

In [34]:
# for better line detection, we want to change the image to black and white, so the line is black, and other pixels are white

# define a zero numpy array in the same size as the image
thresh = np.zeros_like(image)

# first set all the pixels to 255 to make them white
thresh[:,:] = 255

# the pixel_value of the line is equal to 80, so those pixel with the value of 80 set to 0 to make them black
line_pixel_value = 80
thresh[image == line_pixel_value] = 0

# applying adge detection to the changed processed image
thresh = cv2.Canny(thresh, 50, 150, apertureSize=3)

### **Applying Hough Transform to processed image**

In [35]:
# set the line to none first to go to the while loop
lines = None

""" set the value of threshold to 50 -- note that for different images with different quality we must set the specific threshold,
        because if the threshold cant apply hough transform, the line return None, so we must decrease the value of threshold """
threshold = 50

# while loop to apply Hough transform
while lines is None:
   threshold = threshold - 5
   lines = cv2.HoughLines(thresh, rho=1, theta=np.pi/180, threshold=threshold)

### **Calculate the slope and the angle of detected lines**

In [36]:
# define blank lists to save angle and slope of detected lines 
angles = []
slopes = []

for line in lines:
   rho, theta = line[0]
   a = np.cos(theta)
   b = np.sin(theta)
   slope = (-a) / b
   intercept = rho / b
   angle = np.arctan(slope) * 180 / np.pi
   angles.append(angle)
   slopes.append(slope)

# calculate the mean angle to send to agent -- to use mean() we convert the array to numpy array first
angles = np.array(angles)
mean_angle = angles.mean()
print(mean_angle)

72.99998950939658


### **Showing detected lines on the main image**

In [37]:
for line in lines:
  rho, theta = line[0]
  a = np.cos(theta)
  b = np.sin(theta)
  m = (-a) / b
  n = rho / b
  angle = np.arctan(m) * 180 / np.pi
  print(f"rho: {rho}, theta: {theta}, slope: {m}, intercept: {n}, angle: {angle}")
  x0 = a * rho
  y0 = b * rho
  x1 = int(x0 + 1000 * (-b))
  y1 = int(y0 + 1000 * (a))
  x2 = int(x0 - 1000 * (-b))
  y2 = int(y0 - 1000 * (a))
  cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)


# display the original image with the detected lines
cv2.imshow('image with detected lines',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

rho: -259.0, theta: 2.844886541366577, slope: 3.2708511352539062, intercept: -885.8582153320312, angle: 72.99998950939658
rho: -284.0, theta: 2.844886541366577, slope: 3.2708511352539062, intercept: -971.3657836914062, angle: 72.99998950939658
